# 5 - Weather Database Access
***
2023.06.14
***

**Requirements:**
- Internet access
- NSRDB API key. API keys are free and can be requested in about 5 minutes. To get your own key, visit https://developer.nrel.gov/signup/
- Step **1.** is for Eagle HPC users. You will need an account with NREL's Eagle computer for this method.

**Objectives:**

Using direct access to large scale weather databases, we're going to estimate the minimum standoff distance for a roof mounted PV system. We'll do this in 3 ways using both the NSRDB and PVGIS database.
1. Single Location, NSRDB via NREL's high performance computer, Eagle.
2. Single Location via NSRDB public API key.
3. Single Location via the PVGIS public database

**Background:**

This journal will demonstrate all existing built-in methods for directly accessing public weather databases. Some methods are restriced to certain user groups. For general users, see methods **2** and **3**. For users with an active Eagle HPC account, you may use methods **1**, **2**, and **3**.

For all users and all steps: This journal will run significantly longer than other tutorials and have significant internet traffic as you fetch large datasets.

This example demonstrates the calculation of a minimum standoff distance necessary for roof-mounted PV modules to ensure that the $T_{98}$ operational temperature remains under 70°C, in which case the more rigorous thermal stability testing requirements of IEC TS 63126 would not needed to be considered. We use data from [Fuentes, 1987] to model the approximate exponential decay in temperature, $T(X)$, with increasing standoff distance, $X$, as,

$$ X = -X_0 \ln\left(1-\frac{T_0-T}{\Delta T}\right)$$

where $T_0$ is the temperature for $X=0$ (insulated back) and $\Delta T$ is the temperature difference between an insulated back ($X=0$) and open rack mounting configuration ($X=\infty)$.

The following figure showcases this calulation for the entire United States. We used pvlib and data from the National Solar Radiation Database (NSRDB) to calculate the module temperatures for different mounting configuration and applied our model to obtain the standoff distance for roof-mounted PV systems.

# Single location example

In [1]:
import pvdeg

## 1. NSRDB - HSDS on Eagle

This method requires a direct connection to NREL's high performance computer "Eagle". If you are not running this journal from Eagle, skip this section and proceed to section **2.**

In this step:

First we select a database. Here, we will use the NSRDB. Since we are modeling a single location, we can pass the `weather_id` as tuple (lat, long). A location gid can be used as well. 'gid' is a unique identifier to a geographic location within the NSRDB. We'll look at how to find gids later on.

Next, we want to select a satellite, named dataset (year of data), and what weather attributes we want to fetch. For further options, see the documentation for `pvdeg.weather.get`

In [2]:
#Get weather data
weather_db = 'NSRDB'
weather_id = (39.741931, -105.169891)
#weather_id = 1933572  
weather_arg = {'satellite' : 'GOES', 
               'names' : 2021, 
               'NREL_HPC' : True,
               'attributes' : ['air_temperature', 'wind_speed', 'dhi', 'ghi', 'dni','relative_humidity']}

weather_df, meta = pvdeg.weather.get(weather_db, weather_id, **weather_arg)

{'x': 3.4904744924590694, 'T98_0': 82.57377792878141, 'T98_inf': 53.716463725092396}


`pvdeg.weather.get` returns the same variables as `weather.read` which we have used in each journal before this. We get a weather DataFrame and a meta-data dicitonary. Each contains a minimum of consistent fields, but may have additional fields based on the database accessed or the attributes requested.

Lets verify the weather data we fetched by running a familiar calculation; standoff distance.

In [ ]:
res = pvdeg.standards.calc_standoff(weather_df, meta, tilt=None, azimuth=180, sky_model='isotropic', temp_model='sapm', 
                    module_type='glass_polymer', level=1, x_0=6.1)
print(res)

## 2. NSRDB - API

As of now, this method does not work for calculations which require relative humidity.

To access the NREL NSRDB, you will need an API key. Key's are free, but require you to set up an account. Without an API key, you can use a demonstration API which is severely limited. To set up an account and get your API key, visit https://developer.nrel.gov/signup/

Key Notes:
- set `attributes = []` to return all possible attributes (weather fields)

In [2]:
API_KEY = 'your_api_key_here' 
# The example API key here is for demonstation and is rate-limited per IP. 
# To get your own API key, visit https://developer.nrel.gov/signup/

weather_db = 'PSM3'
weather_id = (39.741931, -105.169891)
weather_arg = {'api_key': API_KEY,
               'email': 'user@mail.com',
               'names' : 'tmy',
               'attributes' : [], 
               'map_variables': True}

weather_df, meta = pvdeg.weather.get(weather_db, weather_id, **weather_arg)

#Perform calculation
res = pvdeg.standards.calc_standoff(weather_df, meta, tilt=None, azimuth=180, sky_model='isotropic', temp_model='sapm', 
                    module_type='glass_polymer', level=1, x_0=6.1)
print(res)

-- WARNING --
The API for PSM3 does not currently support relative humidity.
The field "relative_humidity" will be missing from this dataframe.
{'x': 2.0063800910065974, 'T98_0': 77.85091991839371, 'T98_inf': 49.84149463751572}


In [5]:
meta

{'Source': 'NSRDB',
 'Location ID': '145809',
 'City': '-',
 'State': '-',
 'Country': '-',
 'Time Zone': -7,
 'Local Time Zone': -7,
 'Dew Point Units': 'c',
 'DHI Units': 'w/m2',
 'DNI Units': 'w/m2',
 'GHI Units': 'w/m2',
 'Temperature Units': 'c',
 'Pressure Units': 'mbar',
 'Wind Direction Units': 'Degrees',
 'Wind Speed Units': 'm/s',
 'Surface Albedo Units': 'N/A',
 'Version': '3.2.0',
 'latitude': 39.73,
 'longitude': -105.18,
 'altitude': 1820}

## 3. PVGIS

This method uses the PVGIS database, a public resource. It requires no API key or user account.

In [4]:
weather_db = 'PVGIS'
weather_id = (39.741931, -105.169891)
#weather_id = (48.667, 15.885)
#weather_arg = {'map_variables': True}

#TMY
weather_df, meta = pvdeg.weather.get(weather_db, weather_id)

#Perform calculation
# res = pvdeg.standards.calc_standoff(weather_df, meta, tilt=None, azimuth=180, sky_model='isotropic', temp_model='sapm', 
#                     module_type='glass_polymer', level=1, x_0=6.1)
# print(res)